<center>
    <h1>An Introduction to SparkML</h1>
</center>

---

In [1]:
!git clone https://github.com/wchill/HMP_Dataset

Cloning into 'HMP_Dataset'...
remote: Enumerating objects: 865, done.
remote: Total 865 (delta 0), reused 0 (delta 0), pack-reused 865
Receiving objects: 100% (865/865), 1010.96 KiB | 0 bytes/s, done.


In [4]:
!ls -l HMP_Dataset/

total 132
drwxr-x--- 2 dsxuser dsxuser  4096 Jun  1 18:18 Brush_teeth
drwxr-x--- 2 dsxuser dsxuser 12288 Jun  1 18:18 Climb_stairs
drwxr-x--- 2 dsxuser dsxuser  4096 Jun  1 18:18 Comb_hair
drwxr-x--- 2 dsxuser dsxuser  4096 Jun  1 18:18 Descend_stairs
drwxr-x--- 2 dsxuser dsxuser 12288 Jun  1 18:18 Drink_glass
drwxr-x--- 2 dsxuser dsxuser  4096 Jun  1 18:18 Eat_meat
drwxr-x--- 2 dsxuser dsxuser  4096 Jun  1 18:18 Eat_soup
-rw-r----- 1 dsxuser dsxuser   479 Jun  1 18:18 final.py
drwxr-x--- 2 dsxuser dsxuser 12288 Jun  1 18:18 Getup_bed
-rw-r----- 1 dsxuser dsxuser  1610 Jun  1 18:18 impdata.py
drwxr-x--- 2 dsxuser dsxuser  4096 Jun  1 18:18 Liedown_bed
-rw-r----- 1 dsxuser dsxuser  4314 Jun  1 18:18 MANUAL.txt
drwxr-x--- 2 dsxuser dsxuser 12288 Jun  1 18:18 Pour_water
-rw-r----- 1 dsxuser dsxuser  3157 Jun  1 18:18 README.txt
drwxr-x--- 2 dsxuser dsxuser 12288 Jun  1 18:18 Sitdown_chair
drwxr-x--- 2 dsxuser dsxuser 12288 Jun  1 18:18 Standup_chair
drwxr-x--- 2 dsxuser dsxuser  4096 Jun 

In [6]:
!pip install pyspark

     |████████████████████████████████| 217.8MB 138kB/s  eta 0:00:01
     |████████████████████████████████| 204kB 42.8MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [24]:
import os
import glob
import time

In [27]:
try:
    from pyspark.sql import SparkSession
    from pyspark import SparkContext, SparkConf
except ImportError as e:
    raise ImportError('PySpark is not Configured')
    
# Creating a Spark-Context
sc = SparkContext.getOrCreate(SparkConf().setMaster('local[*]'))
# Spark Builder
spark = SparkSession.builder.getOrCreate()

In [50]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, IntegerType

from pyspark.ml.feature import (
        Normalizer,
        StringIndexer,
        OneHotEncoder,
        VectorAssembler
    )

In [9]:
schema = StructType((
        StructField('x', IntegerType(), nullable = True),
        StructField('y', IntegerType(), nullable = True),
        StructField('z', IntegerType(), nullable = True)
    ))

In [17]:
folderList = [i for i in os.listdir('HMP_Dataset') if '.' not in i]

In [28]:
%%time
data = None

for category in folderList:
    print(f'[{time.ctime()}] Reading Category : {category}')
    for dataFile in glob.glob(f'HMP_Dataset/{category}/*.txt'):
        _temp = spark.read.option('header', 'false').option('delimiter', ' ').csv(dataFile, schema = schema)
        
        # Adding the Category & Source-File Information into the Temporary Created DataFrame
        _temp = _temp.withColumn('class', lit(category))
        _temp = _temp.withColumn('source', lit(dataFile))
        
        if data is None:
            data = _temp # Add the Data, for the First Iteration
        else:
            data = data.union(_temp)

[Mon Jun  1 18:39:31 2020] Reading Category : Liedown_bed
[Mon Jun  1 18:39:43 2020] Reading Category : Pour_water
[Mon Jun  1 18:39:58 2020] Reading Category : Descend_stairs
[Mon Jun  1 18:40:04 2020] Reading Category : Standup_chair
[Mon Jun  1 18:40:17 2020] Reading Category : Brush_teeth
[Mon Jun  1 18:40:18 2020] Reading Category : Sitdown_chair
[Mon Jun  1 18:40:32 2020] Reading Category : Climb_stairs
[Mon Jun  1 18:40:45 2020] Reading Category : Eat_meat
[Mon Jun  1 18:40:46 2020] Reading Category : Drink_glass
[Mon Jun  1 18:41:00 2020] Reading Category : Getup_bed
[Mon Jun  1 18:41:15 2020] Reading Category : Eat_soup
[Mon Jun  1 18:41:16 2020] Reading Category : Walk
[Mon Jun  1 18:41:31 2020] Reading Category : Comb_hair
[Mon Jun  1 18:41:36 2020] Reading Category : Use_telephone
CPU times: user 2.39 s, sys: 955 ms, total: 3.35 s
Wall time: 2min 7s


In [30]:
data.show()

+---+---+---+-----------+--------------------+
|  x|  y|  z|      class|              source|
+---+---+---+-----------+--------------------+
| 14| 39| 35|Liedown_bed|HMP_Dataset/Liedo...|
| 13| 39| 36|Liedown_bed|HMP_Dataset/Liedo...|
| 12| 38| 36|Liedown_bed|HMP_Dataset/Liedo...|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|
| 13| 40| 36|Liedown_bed|HMP_Dataset/Liedo...|
| 14| 39| 36|Liedown_bed|HMP_Dataset/Liedo...|
| 13| 38| 34|Liedown_bed|HMP_Dataset/Liedo...|
| 13| 39| 34|Liedown_bed|HMP_Dataset/Liedo...|
| 14| 37| 34|Liedown_bed|HMP_Dataset/Liedo...|
| 13| 39| 34|Liedown_bed|HMP_Dataset/Liedo...|
| 13| 39| 34|Liedown_bed|HMP_Dataset/Liedo...|
| 15| 37| 34|Liedown_bed|HMP_Dataset/Liedo...|
| 14| 37| 34|Liedown_bed|HMP_Dataset/Liedo...|
| 15| 42| 35|Liedown_bed|HMP_Dataset/Liedo...|
| 14| 39| 33|Liedown_bed|HMP_Dataset/Liedo...|
| 13| 37| 33|Liedown_bed|HMP_Dataset/Liedo...|
| 10| 39| 31|Liedown_bed|HMP_Dataset/Liedo...|
| 10| 40| 30|

In [34]:
indexer = StringIndexer(inputCol = 'class', outputCol = 'classIndex')

In [35]:
indexedData = indexer.fit(data).transform(data)

In [36]:
indexedData.show()

+---+---+---+-----------+--------------------+----------+
|  x|  y|  z|      class|              source|classIndex|
+---+---+---+-----------+--------------------+----------+
| 14| 39| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 13| 39| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 12| 38| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 13| 40| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 14| 39| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 13| 38| 34|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 13| 39| 34|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 14| 37| 34|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 13| 39| 34|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 13| 39| 34|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 15| 37| 34|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 14| 37| 34|Liedown_bed|HMP_Dataset/Liedo...|      12.0|
| 15| 42| 35|L

In [38]:
encoder = OneHotEncoder(inputCol = 'classIndex', outputCol = 'categoryVector')

In [40]:
encodedData = encoder.transform(indexedData)

In [41]:
encodedData.show()

+---+---+---+-----------+--------------------+----------+---------------+
|  x|  y|  z|      class|              source|classIndex| categoryVector|
+---+---+---+-----------+--------------------+----------+---------------+
| 14| 39| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|
| 13| 39| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|
| 12| 38| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|
| 13| 40| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|
| 14| 39| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|
| 13| 38| 34|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|
| 13| 39| 34|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|
| 14| 37| 34|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|
| 13| 39| 34|Liedown_bed|HMP_Dataset/L

In [43]:
assembler = VectorAssembler(inputCols = 'x y z'.split(), outputCol = "features")
features  = assembler.transform(encodedData)

In [44]:
features.show()

+---+---+---+-----------+--------------------+----------+---------------+----------------+
|  x|  y|  z|      class|              source|classIndex| categoryVector|        features|
+---+---+---+-----------+--------------------+----------+---------------+----------------+
| 14| 39| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[14.0,39.0,35.0]|
| 13| 39| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,39.0,36.0]|
| 12| 38| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[12.0,38.0,36.0]|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,38.0,35.0]|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,38.0,35.0]|
| 13| 40| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,40.0,36.0]|
| 14| 39| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[14.0,39.0,36.0]|
| 13| 38| 34|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,38.0,34.0]|

In [46]:
StdScalar = Normalizer(inputCol = 'features', outputCol = 'NormalizedFeatures', p = 1.0)

In [48]:
NormalizedData = StdScalar.transform(features)

In [49]:
NormalizedData.show()

+---+---+---+-----------+--------------------+----------+---------------+----------------+--------------------+
|  x|  y|  z|      class|              source|classIndex| categoryVector|        features|  NormalizedFeatures|
+---+---+---+-----------+--------------------+----------+---------------+----------------+--------------------+
| 14| 39| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[14.0,39.0,35.0]|[0.15909090909090...|
| 13| 39| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,39.0,36.0]|[0.14772727272727...|
| 12| 38| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[12.0,38.0,36.0]|[0.13953488372093...|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,38.0,35.0]|[0.15116279069767...|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,38.0,35.0]|[0.15116279069767...|
| 13| 40| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,40.0,36.0]|[0.14606741573

In [54]:
ETL_PipeLine = Pipeline(stages = [
    indexer,   # Indexing Data, using StringIndexer
    encoder,   # One-Hot Encoding of the Data
    assembler, # Assemble Data as Vector Objects
    StdScalar  # Scaling the Data, i.e. Normalization
])

model      = ETL_PipeLine.fit(data)
prediction = model.transform(data)

In [55]:
prediction.show()

+---+---+---+-----------+--------------------+----------+---------------+----------------+--------------------+
|  x|  y|  z|      class|              source|classIndex| categoryVector|        features|  NormalizedFeatures|
+---+---+---+-----------+--------------------+----------+---------------+----------------+--------------------+
| 14| 39| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[14.0,39.0,35.0]|[0.15909090909090...|
| 13| 39| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,39.0,36.0]|[0.14772727272727...|
| 12| 38| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[12.0,38.0,36.0]|[0.13953488372093...|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,38.0,35.0]|[0.15116279069767...|
| 13| 38| 35|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,38.0,35.0]|[0.15116279069767...|
| 13| 40| 36|Liedown_bed|HMP_Dataset/Liedo...|      12.0|(13,[12],[1.0])|[13.0,40.0,36.0]|[0.14606741573

In [56]:
trainingData = prediction.drop('x').drop('y').drop('z').drop('class').drop('classIndex').drop('features')

In [57]:
trainingData.show()

+--------------------+---------------+--------------------+
|              source| categoryVector|  NormalizedFeatures|
+--------------------+---------------+--------------------+
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.15909090909090...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.14772727272727...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.13953488372093...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.15116279069767...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.15116279069767...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.14606741573033...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.15730337078651...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.15294117647058...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.15116279069767...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.16470588235294...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.15116279069767...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.15116279069767...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0.17441860465116...|
|HMP_Dataset/Liedo...|(13,[12],[1.0])|[0